In [2]:
import pandas as pd
import numpy as np
import polars as pl

import plotly.express as px

import pydsp.run.worker
import pydsptools.biorad.parse as bioradparse
import pydsptools.plot as dspplt
import pprint
import pyarrow as pa
import os
import subprocess
import matplotlib.pyplot as plt
from pathlib import Path

# Background

baseline fitting 알고리즘이 CFX manager, DSP-Legacy, and AutoBaseline 등과 같이 여러 종류가 있고 그 fitting 성능이 직관적으로 보기에 개선의 필요성을 느낌.
이 프로젝트를 통해서 각 각의 baseline 알고리즘의 장/단점을 도출하고 더 향상된 baseline fitting알고리즘을 개발하는 것을 이 프로젝트의 목적으로 한다.

* 프로젝트 발의자: 손형석 차장
* 프로젝트 담당자: 김광민 대리
* 문서 작성자: 김광민 대리 

현 시점 (2024-02-05)에서 구체적인 목표 및 action items이 부재한 추상적인 상태이다. 

# Goals

회의 (2024-02-06) 결과, 다음과 같은 일시적인 목표를 정함.

* insights 도출과 직관적인 분석을 위해 3개의 알고리즘의 fitting 결과를 시각화한다. 
* 시각화에 필요한 분석 환경을 마련한다.
    * 기제품 raw data 확보, 전처리하여 dsp 연산
    * 기제품 data baseline-subtracted by cfx manager 확보, 전처리하여 dsp 연산
    * 기제품 auto-baseline 처리된 data 확보 
    * data 병합 및 시각화


# Script Description

* 이 스크립트는 실무진들 업무 파악 및 정리가 이루어지기전에 작업한 코드도 섞여 있다.
    * (채택되지 않은 방식): 업무 파악 전 코드 
    * (채택된 방식): 업무 파악 후 코드
* `1.cfx-to-parquet-converter.ipynb` 는 다음 2가지의 data를 전처리한다.
    * CFX Data named  (RFU Baseline Subtracted by CFX Manager) 를 `bioradparse.load_pcrdata()` 함수에 의해 전처리된 결과물인 parquet 파일로 변환한다.
    * Raw Sample Data (sampled from `pda-raw-PRJDS001`) 를 `bioradparse.load_pcrdata()` 함수에 의해 전처리된 결과물인 parquet 파일로 변환한다.
* 최종 결과물인 `merged_data` 는 DSP 연산을 돌리기 전 data를 병합시킨 결과물이다. 

## CFX Data (Baseline Subtracted) Parquet Files 생성

* Seegene Export xlsx or batchanalyzer csv 데이터를 Parquet 파일로 변환하기 위한 path 설정


In [3]:
root_path = Path.cwd() # /home/kmkim/pda/dsp-research-strep-a/kkm
prefix = 'data'
directory_names = ['cfx-baseline-subtracted','pda-raw-sample']
product_names = ['GI-B-I', 'GI-B-II', 'GI-P', 'GI-V', 'RP1', 'RP2', 'RP3', 'RP4', 'STI-CA', 'STI-EA', 'STI-GU']
consumables = ['8-strip','96-cap']
plate_numbers = ['plate_data_' + number for number in ['002','005','031','032','036','041']]
# !python -m pydsptools.biorad.parse -t cfx-batch-csv -f parquet -o './data/baseline-subtracted/processed/example1' './data/baseline-subtracted/cfx-data/'

### 1. Parquet Files을 `plate_data_{number}` path에 분산 배치 방식
* (채택되지 않은 방식) : data paths가 복잡해짐
* `bioradparse.load_pcrdata()`를 이용한 batchanalyzer csv 데이터를 Parquet 파일로 변환 (아래 코드 돌릴 필요 없음- 아래의 다른 방식으로 돌릴 예정) 
* 이 방식은 실무진들이 업무 파악 및 정리가 되지 않아 임시 시도한 방식

In [65]:
# cfx_data = []
# raw_data = []
# 
# for directory_name in directory_names: 
#     for product_name in ['GI-B-I']: #product_names:
#         for consumable in ['8-strip']: #consumables:
#             for plate_number in plate_numbers:
#                 full_path = root_path / prefix / directory_name / product_name / consumable / plate_number
#                 processed_path = full_path / "processed" / "example1"
#                 processed_path.mkdir(parents=True, exist_ok=True)
#                 exporting_path =  full_path / "exported_pcrd"
#                 if 'cfx' in exporting_path: 
#                     temp_cfx_data = bioradparse.load_pcrdata(exporting_path, datatype="cfx-xl")
#                     cfx_data.append(temp_cfx_data)
#                 temp_raw_data = bioradparse.load_pcrdata(exporting_path, datatype="cfx-batch-csv")
#                 raw_data.append(temp_raw_data)
# #pathlib.Path(f"./data/baseline-subtracted/processed/example1")
# 
# for pcrname, pcrdata in raw_data.items():
#     bioradparse.save_pcrdata(raw_data, root_path / "pda-raw-sample" / "processed" / "example1" / f"{pcrname}.parquet")
# for pcrname, pcrdata in cfx_data.items():
#     bioradparse.save_pcrdata(cfx_data, root_path / "cfx-baseline-subtraction" / "processed" / "example1" / f"{pcrname}.parquet")

### 2. Parquet Files을 `./data/cfx-baseline-subtracted/cfx-data` 에 batch로 배치
* (채택된 방식)
* `batchanalyzer.exe`로 CFX Manager Baseline Subtracted Data 대량 batch 추출 후 `directory-path/cfx-data`에 저장  bioradparse module 돌림
* 아래 코드 한번만 돌리면 됨

In [4]:
cfx_datapath = root_path / prefix / directory_names[0] / 'cfx-data'
cfx_data = bioradparse.load_pcrdata(str(cfx_datapath), datatype="cfx-batch-csv") # output: dictionary

In [5]:
len(cfx_data.keys()) # 201개 plates

# Convert PyArrow Tables to DataFrames and store them in a new dictionary
cfx_df_dict = {key: value.to_pandas() for key, value in cfx_data.items()}

# Convert the dictionary of DataFrames to a single DataFrame (concatenating along rows)
cfx_df = pd.concat(cfx_df_dict.values(), axis=0)
#cfx_df = cfx_df.rename(columns={'rfu':'cfx_rfu','endrfu':'cfx_endrfu','melt_idx':'cfx_melt_idx','melt':'cfx_melt'})
cfx_df['combo_key'] = cfx_df.apply(lambda x: f"{x['name']} {x['channel']} {x['step']} {x['well']} {x['welltype']}", axis=1)



In [7]:
# export the preprocessed dataframe 
cfx_df.to_parquet('./data/cfx-baseline-subtracted/merge_cfx-baseline-subtracted_kkm_v1_20240213.parquet', index=False)

## Raw Sample Data 생성
* `/home/kmkim/pda/dsp-research-strep-a/kkm/data/pda-raw-PRJDS001/` 기제품 데이터 중 `GI-B-I/GI-B-I_8-strip` 일부를 sampling함
* sampling 방식은 아래의 criteria를 갖고 manual sampling
    * Incusion criteria: 다음과 같은 조건을 만족시키는 신호 선별 
        * Baseline 차감 전 일정한 pattern 띄는 음성 신호 
        * Baseline 차감 후 음성 신호에서 다른 pattern이 보이는 plate 
        * RFU magnitude 120 이상 
        * Baseline 차감 후 신호의 Main pattern과 다른 pattern 보이는 plate
            * 예) 하향 평행이동한 noise vs V-shape pattern
    * Exclusion criteria: 차감 전 pattern과 차감 후 pattern이 유사한 신호 

In [8]:
for plate_number in plate_numbers:
    directory_path_raw = root_path / prefix / directory_names[1] / product_names[0] / f"{product_names[0]}_{consumables[0]}"
    raw_datapath = directory_path_raw / plate_number
    input_path = str(raw_datapath)+ '/exported_pcrd'
    temp_raw_data = bioradparse.load_pcrdata(input_path, datatype="cfx-batch-csv")
    raw_df_dict = {key: value.to_pandas() for key, value in temp_raw_data.items()}
    raw_df = pd.concat(raw_df_dict.values(), axis=0)
raw_df['combo_key'] = raw_df.apply(lambda x: f"{x['name']} {x['channel']} {x['step']} {x['well']} {x['welltype']}", axis=1)

In [9]:
# export the preprocessed dataframe 
raw_df.to_parquet('./data/pda-raw-sample/merge_pda-raw-sample_kkm_v1_20240213.parquet', index=False)

## Merged Data

CFX data와 Raw Sample Data 병합

In [38]:
data = pd.merge(cfx_df,raw_df[['rfu','endrfu','combo_key']],on='combo_key',how='inner')
print(data.columns)
print(data.head)

Index(['name', 'step', 'channel', 'well', 'welltype', 'cfx_rfu', 'cfx_endrfu',
       'cfx_melt_idx', 'cfx_melt', 'combo_key', 'rfu', 'endrfu'],
      dtype='object')
<bound method NDFrame.head of                                                   name  step      channel  \
0    admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     4  Cal Red 610   
1    admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     4  Cal Red 610   
2    admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     4  Cal Red 610   
3    admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     4  Cal Red 610   
4    admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     4  Cal Red 610   
..                                                 ...   ...          ...   
763  admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     5   Quasar 670   
764  admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     5   Quasar 670   
765  admin_2015-03-23 18-23-41_CC010436_Allplex GI ...     5   Quasar 670   
766  admin_2015-03-23 18-23-41_CC